In [2]:
import pickle
pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("y_test.pickle","rb")
y_test = pickle.load(pickle_in)


In [4]:
print(y_test.shape,X_train.shape)
print(X_test.shape,y_train.shape)

(1050,) (4200, 100, 100, 1)
(1050, 100, 100, 1) (4200,)


In [5]:
trainX, testX = X_train, X_test
trainy, testy = y_train, y_test
print(trainy.shape, testy.shape)

(4200,) (1050,)


In [7]:
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import plot_model
from keras.models import load_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from numpy import argmax

In [12]:
# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'modelss/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models
 

In [45]:
# define stacked model from multiple member input models
def define_stacked_model(members):
	# update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	hidden = Dense(64, activation='relu')(merge)
	output = Dense(1, activation='softmax')(hidden)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
	return model

In [49]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
    #inputy=np.asarray(inputy).astype('float32').reshape((-1,1))
	#inputy=np.asarray(inputy).astype('float32').reshape((-1,1))
	model.fit(X, inputy, epochs=30)

In [50]:
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [51]:
import numpy as np
# load all models
n_members = 3
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# define ensemble model
stacked_model = define_stacked_model(members)
# fit stacked model on test dataset
fit_stacked_model(stacked_model, testX, testy)
# make predictions and evaluate
yhat = predict_stacked_model(stacked_model, testX)
yhat = argmax(yhat, axis=1)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

>loaded modelss/model_1.h5
>loaded modelss/model_2.h5
>loaded modelss/model_3.h5
Loaded 3 models
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Epoch 1/30
33/33 [==============================] - 99s 3s/step - loss: 0.6436 - accuracy: 0.4048
Epoch 2/30
33/33 [==============================] - 95s 3s/step - loss: 0.5889 - accuracy: 0.4048
Epoch 3/30
33/33 [==============================] - 95s 3s/step - loss: 0.5387 - accuracy: 0.4048
Epoch 4/30
33/33 [==============================] - 95s 3s/step - loss: 0.4965 - accuracy: 0.4048
Epoch 5/30
33/33 [==============================] - 96s 3s/step - loss: 0.4661 - accuracy: 0.4048
Epoch 6/30
33/33 [==============================] - 97s 3s/step - loss: 0.4480 - accuracy: 0.4048
Epoch 7/30
33/33 [==============================] - 97s 3s/step - loss: 0.4379 - accuracy: 0.4048
Epoch 8/30
33/33 [=============================